In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
years = list(range(2023, 2016, -1))
all_matches = []

In [3]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = shooting[["Date", "Sh", "SoT", "G/Sh","G/SoT","Dist", "FK", "PK", "PKatt"]]
        except ValueError as e:
            print(e)
            break
        
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(3)

C:\Users\dayvi\AppData\Local\Temp\ipykernel_17880\2990487697.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\dayvi\AppData\Local\Temp\ipykernel_17880\2990487697.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Team"] = team_name
C:\Users\dayvi\AppData\Local\Temp\ipykernel_17880\2990487697.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\dayvi\AppData

In [4]:
shooting = pd.concat(all_matches)
shooting.columns = [c.lower() for c in shooting.columns]

In [5]:
dict_shooting = shooting.to_dict(orient="records")

In [6]:
from database import create_documents

In [7]:
documents = create_documents('shooting', dict_shooting)